In [ ]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)


In [ ]:
# all modules necessary for this nb
import os
import sys
import pickle

import numpy as np
import pylab as pl

# setting parameters for default matplotlib plots
%matplotlib inline

In [ ]:
pl.style.use('fabiostyle')
# needs to find the library of functions
sys.path.append('../../../../code/')  # to be replaced!

import utils as ut
import plots as pt

In [ ]:
NOTEBOOK_NAME = 'patterns_time_bins'

In [ ]:
from pickleshare import PickleShareDB

autorestore_folder = os.path.join(os.getcwd(), 'autorestore', NOTEBOOK_NAME)
db = PickleShareDB(autorestore_folder)
import sys
from workspace import *
import IPython
ip = IPython.get_ipython()

# this will restore all the saved variables. ignore the errors listed.
load_workspace(ip, db)

# use `save_worspace(db)` to save variables at the end

In [ ]:
sessions = 'post',#'expo','d1','d3'

In [ ]:
#import variables.
for sess in sessions:
    try:
        exec("folder = '../%s/notebooks/autorestore/preprocessing'")%sess
        for var in ('time_ax', 'traces', 'events', 'cycles',
                'is_STIM5t', 'is_STIM6t', 'is_STIM7t','is_consumed_STIM5t','is_CSm_enabledt',
                'rewards','shocks','licks','punished','avoided','enabled',
                'CSm_enabled','armed','is_rewardedt','is_shockedt',
                'is_avoidedt','consumption_times','CYCLE_START','CS_DURATION','DELAY','US_START'): 
            exec('%s_%s = ut.load_variable(\'%s\', folder=\'%s\')'%(var, sess, var, folder))
    except:
        print ('%s failed')%sess

In [ ]:
#this is how data are extracted in the cellregs nb
import h5py
try:
    filepath = '../data/cellreg_pre_vs_post.mat'
    arrays = {}
    f = h5py.File(filepath)
    cellreg_map = np.array(f['cell_registered_struct']['cell_to_index_map'])-1
    keep = np.prod(cellreg_map, 0)>=0
    cellregs_pre_vs_post = dict(zip(['pre', 'post'], cellreg_map.T[keep].T.astype(int)))
except:
    print 'failed pre_vs_post'

In [ ]:
try:
    conditions = ['pre', 'post']
    cellregs_all = ut.load_variable('cellregs_pre_vs_post', './autorestore/cellregs')
except:
    print 'failed'

In [ ]:
try:
    conditions = ['expo', 'pre']
    cellregs_all = ut.load_variable('cellregs_pre_vs_expo', './autorestore/cellregs')
except:
    print 'failed'

In [ ]:
try:
    conditions = ['expo', 'post']
    cellregs_all = ut.load_variable('cellregs_expo_vs_post', './autorestore/cellregs')
except:
    print 'failed'

In [ ]:
#0.5sec time bins
delta_t0 = 0.5
t_start0 = -3  # sec
t_stop0 = 11.5  # sec

for sess in sessions:
    try:
        exec("patterns_evs_05s_%s = np.r_[[ut.extract_activity(time_ax_%s,events_%s, cycles_%s,\
                                       CYCLE_START_%s,i*delta_t0, (i+1)*delta_t0, offset=t_start0)\
                                       for i in xrange(int((t_stop0-t_start0)/delta_t0))]]"%(sess,sess,sess,sess,sess))
    except:
        print 'nope %s'%(sess)

In [ ]:
#1sec time bins
delta_t0 = 1
t_start0 = -3  # sec
t_stop0 = 11  # sec

for sess in sessions:
    try:
        exec("patterns_evs_1s_%s = np.r_[[ut.extract_activity(time_ax_%s,events_%s, cycles_%s,\
                                       CYCLE_START_%s,i*delta_t0, (i+1)*delta_t0, offset=t_start0)\
                                       for i in xrange(int((t_stop0-t_start0)/delta_t0))]]"%(sess,sess,sess,sess,sess))
    except:
        print 'nope %s'%(sess)

In [ ]:
#1sec time bins centered on the half-second
delta_t0 = 1
t_start0 = -3.5  # sec
t_stop0 = 10.5  # sec

for sess in sessions:
    try:
        exec("patterns_evs_15s_%s = np.r_[[ut.extract_activity(time_ax_%s,events_%s, cycles_%s,\
                                       CYCLE_START_%s,i*delta_t0, (i+1)*delta_t0, offset=t_start0)\
                                       for i in xrange(int((t_stop0-t_start0)/delta_t0))]]"%(sess,sess,sess,sess,sess))
    except:
        print 'nope %s'%(sess)

In [ ]:
#2sec time bins
delta_t0 = 2
t_start0 = -2  # sec
t_stop0 = 10  # sec

for sess in sessions:
    try:
        exec("patterns_evs_2s_%s = np.r_[[ut.extract_activity(time_ax_%s,events_%s, cycles_%s,\
                                       CYCLE_START_%s,i*delta_t0, (i+1)*delta_t0, offset=t_start0)\
                                       for i in xrange(int((t_stop0-t_start0)/delta_t0))]]"%(sess,sess,sess,sess,sess))
    except:
        print 'nope %s'%(sess)

In [ ]:
#replace all nan values with 0
for tb in ['05s','1s','15s','2s']:
    for sess in sessions:
        try:
            for x in np.argwhere(np.isnan(eval("patterns_evs_%s_%s"%(tb,sess)))):
                print x,
                exec("patterns_evs_%s_%s[x[0],x[1],x[2]]=0"%(tb,sess))
        except:
            print 'nope',tb,sess

In [ ]:
#make sure no more nan values remain (nothing should be printed when run)
for tb in ['05s','1s','15s','2s']:
        for sess in sessions:
            try:
                for x in np.argwhere(np.isnan(eval("patterns_evs_%s_%s"%(tb,sess)))):
                    print x,
            except:
                print 'nope',tb,sess

In [ ]:
for sess in sessions:
    try:
        exec('labels_%s = ((np.r_[is_STIM5t_%s]*1)+(np.r_[is_STIM6t_%s]*2)+(np.r_[is_STIM7t_%s]*3)+(np.r_[is_rewardedt_%s]*3)\
        +(np.r_[is_consumed_STIM5t_%s]*1)+(np.r_[is_CSm_enabledt_%s]*4)+(np.r_[is_avoidedt_%s]*4))'%(sess,sess,sess,sess,sess,sess,sess,sess))
    except:
        print'nope %s'%(sess)

In [ ]:
#trial type labels:
#reward w/o eliciting sucrose = 1, rew eliciting reward but not collecting within 1.5s of delivery=+ = 4, rew w/ consumption = 5
#CSm w/o running > threshold= 2, CSm w/ running > threshold = 6
#AA shocked= 3, AA avoided shock (running > threshold) = 7

In [ ]:
##making list for extracting cellreg comparisons
#CR_comparisons = [['pre','mid'],['pre','post'],['mid','post']]

In [ ]:
##extract patterns using only registered cells
#for tb in ['05s','1s','2s']:
#    for cr in CR_comparisons:
#            try:
#                exec("patterns_evs_reg_%s_%s_%s = patterns_evs_%s_%s[:, :, cellregs_%s_vs_%s['%s']]"%(tb,cr[0],cr[1],
#                                                                  tb,cr[0],cr[0],cr[1]))
#                exec("patterns_evs_reg_%s_%s_%s = patterns_evs_%s_%s_%s[:, :, cellregs_%s_vs_%s['%s']]"%(tb,cr[1],cr[0],
#                                                                  tb,cr[1],cr[0],cr[1]))
#            except:
#                print 'nope %s'%tb,cr,

In [ ]:
save_workspace(db);